# Deploying models using a serverless Azure Function

Method:
1. Train scikit-learn model
2. Create Azure function
3. Create Azure function app and publish

# 1. Train

In [ ]:
import numpy as np
import sklearn
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import joblib

X_train, y_train, X_test, y_test = [np.load("../data/diabetes.npz")[x] for x in ("X_train", "y_train", "X_test", "y_test")]

alpha=0.1

model = Ridge(alpha=alpha).fit(X_train, y_train)

joblib.dump(model, "diabetes_model.pkl")

# 2. Create Azure Function

We need to setup the Azure CLI to get started. First update the CLI:

In [ ]:
!sudo apt-get install --only-upgrade azure-cli

Install Azure Functions CLI package following [the docs](https://docs.microsoft.com/en-us/azure/azure-functions/functions-run-local?tabs=v4%2Clinux%2Ccsharp%2Cportal%2Cbash#v2):

In [ ]:
!curl https://packages.microsoft.com/keys/microsoft.asc | gpg --dearmor > microsoft.gpg
!sudo mv microsoft.gpg /etc/apt/trusted.gpg.d/microsoft.gpg
!sudo sh -c 'echo "deb [arch=amd64] https://packages.microsoft.com/repos/microsoft-ubuntu-$(lsb_release -cs)-prod $(lsb_release -cs) main" > /etc/apt/sources.list.d/dotnetdev.list'
!sudo apt-get update
!sudo apt-get install azure-functions-core-tools-4 -y

Login to Azure CLI. Make sure you are using an account under the correct subscription (change using `--tenant <TENANT_ID>` option).

In [ ]:
!az login

In [ ]:
!az login --tenant f2ec3ef9-cca6-46ec-be61-845d74fcae94

Initialise local function project folder:

In [ ]:
!func init DiabetesInferenceFunctionProject --python

In [ ]:
%cd DiabetesInferenceFunctionProject

Create new function that is triggered by HTTP:

In [ ]:
!func new --name DiabetesInferenceFunction --template "HTTP trigger" --authlevel "anonymous" --worker-runtime python

Copy scikit-learn model and add inferencing script + dependencies. Make sure scikit-learn is same version in which model was created otherwise may get compatibility errors (e.g. see [here](https://github.com/scikit-learn/scikit-learn/issues/24387) for common scikit-learn error and see [here](https://github.com/scikit-image/scikit-image/issues/5060) for a python3.8 vs 3.9 issue for older sklearn versions)

In [ ]:
!cp ../diabetes_model.pkl DiabetesInferenceFunction

In [ ]:
%%writefile -a requirements.txt

numpy
scikit-learn==0.22.1

In [ ]:
%%writefile DiabetesInferenceFunction/__init__.py
import azure.functions as func
import numpy as np
import logging, sklearn, joblib, json

def main(req: func.HttpRequest) -> func.HttpResponse:
    logging.info(sklearn.__version__) # check this matches sklearn version in which model was trained
    pred = joblib.load('DiabetesInferenceFunction/diabetes_model.pkl').predict(np.array(req.get_json()['data']))
    return func.HttpResponse(json.dumps({"prediction": pred.tolist()}))

Optional: test model locally using `func start` in a new terminal and query local API with requests in Test section below.

# 3. Create Azure function app and publish

Create storage account under appropriate resource group (ideally keep all assets under same resource group for cost tracking). From here on we use resource group `data-ai-academy-sept-2022`:

In [ ]:
!az storage account create --name diabetesfunctionstorage2 --resource-group data-ai-academy-sept-2022 --sku Standard_LRS

Create function app under same resource group and location. Check that runtime-version is the same as python version in which model was created/compatible with the sklearn version.

In [ ]:
!az functionapp create --resource-group data-ai-academy-sept-2022 --consumption-plan-location uksouth --runtime python --runtime-version 3.8 --functions-version 4 --name DiabetesInferenceFunctionApp --os-type linux --storage-account diabetesfunctionstorage2

!az functionapp update --resource-group andreww-academy --name DiabetesInferenceFunctionApp

Publish function app (i.e. deploy). If first time results in Deploy Failed, just try rerunning the command (**shrugs shoulders**). Take note of the invoke URL at the end for API queries.

In [ ]:
!func azure functionapp publish DiabetesInferenceFunctionApp

In [ ]:
endpoint_url = "https://diabetesinferencefunctionapp.azurewebsites.net/api/diabetesinferencefunction"

# Test endpoint and delete

In [ ]:
import requests
import json
input_payload = json.dumps({
    'data': X_test[0:2].tolist(),
})

requests.post(endpoint_url, input_payload, headers={'Content-Type':'application/json'}).json()

Delete created resources. Or you could do this in Azure portal.

In [ ]:
!az functionapp delete --name DiabetesInferenceFunctionApp --resource-group data-ai-academy-sept-2022

In [ ]:
!az storage account delete --name diabetesfunctionstorage2 --resource-group data-ai-academy-sept-2022 -y

Optional: delete local files

In [ ]:
%cd ..

In [ ]:
!rm -r DiabetesInferenceFunctionProject

# Manage

You can manage and monitor the logs of the function app on the Azure portal under Function App.

# References

https://docs.microsoft.com/en-us/azure/azure-functions/create-first-function-cli-python?tabs=azure-cli%2Ccmd%2Cbrowser